**Łukasz Folwarczyk**

*"Badanie i implementacja algorytmów detekcji drogi przy wykorzystaniu cyfrowego przetwarzania obrazów."*

---

# Jsons to dataset


Notatnik pozwalający na odtworzenie zdjęć oraz ich etykiet z pliku JSON. Działanie modułu opisane zostało w paragrafie 4.3.

In [1]:
import argparse
import json
import os
import os.path as osp
from IPython.display import clear_output
import PIL.Image
from labelme import utils

In [2]:
def jsons_to_dataset(input_dir, output_dir):
    """ 
    Opis:
    ---
    Funkcja pobiera jako argument nazwę katalogu, w którym zapisane 
    są pliki JSON. W każdym pliku JSON zapisany jest obraz + etykieta + 
    + wizualizacja (obraz z nałożoną etykieta). Funkcja odczytuje te 
    dane i na ich podstawie tworzy trzy pliki z rozszerzeniem png.
    
    Każdy plik zapisywnay jest w jednym z podkatalogów:
    imgs - obrazy oryginalne
    labels - etykiety
    vizs - wizualizacje
    
    Podkatalogi tworzone są w katalogu, którego nazwa stanowi argument 
    (output_dir) funkcji.
    
    Argumenty:
    ---
        input_dir (string):     nazwa katalogu, w którym 
                                zapisane są pliki JSON
        
        input_files (string):   nazwa katalogu wynikowego

    Return:
    ---
        None
    """
    counter = 0
    # odczytanie plików JSON z katalogu wejściowego
    if input_dir[-1] != '/':
        input_dir += '/'   
    
    if output_dir[-1] != '/':
        output_dir += '/'   
    
    json_files_list = os.listdir(input_dir)
    
    # pętla wykonywana dla każdego z plików JSON
    for i in json_files_list:
        json_file = input_dir + i                                       # pełna ścieżka pliku JSON
        data = json.load(open(json_file))                               # otworzenie pliku JSON
        img = utils.img_b64_to_array(data['imageData'])                 # odtworzenie obrazu oryginalnego
        label, label_names = utils.labelme_shapes_to_label(img.shape,   # odtworzenie etykiety
                                                       data['shapes'])
        classes = ['%d: %s' % (l, name) for l,                          # klasy występujące w etykiecie
                    name in enumerate(label_names)]
        label_viz = utils.draw_label(label, img, classes)               # odtworzenie wizualizacji

        imgs_path = os.getcwd() + '/' + output_dir + '/imgs/'           # katalog wyjściowy obrazu
        labels_path = os.getcwd() + '/' + output_dir + '/labels/'       # katalog wyjściowy etykiety
        labels_path_viz = os.getcwd() + '/' + output_dir + '/vizs/'     # katalog wyjściowy wizualizacji

        # stworzenie katalogów wyjściowych 
        if not osp.exists(output_dir):
            os.mkdir(output_dir)
            
        if not osp.exists(imgs_path):
            os.mkdir(imgs_path)

        if not osp.exists(labels_path):
            os.mkdir(labels_path)

        if not osp.exists(labels_path_viz):
            os.mkdir(labels_path_viz)

        # nazwa pliku png (jednakowa dla obrazu, etykiety oraz wizualizacji)
        imgae_path_split = data['imagePath'].split('/')
        image_name = imgae_path_split[-1]
        image_name = image_name[:-4] + '.png'
        
        # ścieżki plików png
        img_path = imgs_path + image_name
        label_path = labels_path + image_name
        viz_path = labels_path_viz + image_name
        
        # zapisanie plików png
        PIL.Image.fromarray(img).save(img_path)
        PIL.Image.fromarray(label).save(label_path)
        PIL.Image.fromarray(label_viz).save(viz_path)

        counter += 1
        clear_output()
        print('Zapisanych: {}/{}'.format(counter, len(json_files_list)))

# Przykład

In [3]:
jsons_path = './JSON_example/input'
output_path =  './JSON_example/output'

In [4]:
jsons_to_dataset(jsons_path, output_path)

Zapisanych: 3/3
